# Homework 3
The goal of the homework was to achieve at least 55% accuracy for image classification on the TinyImagenet-200 dataset. 

Main part of the code is based on the https://github.com/papkov/tinyimagenet.git repository.

The notebook was run on kaggle.com because it is about two times faster than Colab and it is more transparent about the limitations of the environment.

## Initialize runtime

In [6]:
!git clone https://github.com/svnrk/tinyimagenet.git

In [14]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip tiny-imagenet-200.zip -d /kaggle/working/tinyimagenet/data
#!rm tiny-imagenet-200.zip

In [4]:

!pip install hydra-core omegaconf
!pip install -U albumentations

In [17]:
%cd tinyimagenet

In [20]:
import sys
import os
sys.path.insert(0, ".")

from modules import dataset
# you can call reload(module) if you update the source code e.g. with `git pull`
from importlib import reload

import albumentations as albu
import matplotlib.pyplot as plt
from omegaconf import OmegaConf, DictConfig
from pathlib import Path

In [21]:
reload(dataset)
cfg = OmegaConf.load('config/data/tinyimagenet.yaml')

data_root = Path(cfg.root)
train_path = data_root / cfg.train
val_path = data_root / cfg.val

train_dataset = dataset.TinyImagenetDataset(train_path, cfg, None)
val_dataset = dataset.TinyImagenetDataset(val_path, cfg, None)

## Display data example

In [22]:
def clean_show(ax):
    "Show plt figure without axes in tight layout"
    plt.setp(ax, xticks=[], yticks=[])
    plt.tight_layout()
    plt.show()


def show_examples(dataset, n_examples=10, s=2):
    fig, axes = plt.subplots(ncols=n_examples, figsize=(s*n_examples, s))
    for i, (ax, item) in enumerate(zip(axes, dataset)):
        ax.imshow(item.image)
        ax.set_title(item.id)
        ax.set_xlabel(item.label)
    clean_show(axes)

show_examples(train_dataset)

## Augmentation
The changes to the augmentaion were based on the article https://medium.com/@lokeshpara17/tiny-imagenet-using-pytorch-42a3f2ee3c9d
0. input image 64x64
1. Pad to size 72x72
2. RandomCrop to size 64x64
3. Rotate to limit 15deg
4. Horizontal flip 
5. RandomGamma 
6. RandomContrast
7. RandomBrightness
8. Cutout hole (8x8)

## Training


### ResNet18_v2

At first ResNet18_v2 was used to complete the task. Since the evaluation code did not work on custom networks, the result on this work are not complete and other options were looked at. 

In [ ]:
# Command for training custom ResNet18_v2
#!python train.py train.epochs=40 train.batch_size=256

### ResNet50
For the second try torchvisions ResNet50 was used. The "official" network allowed evauation and from that in showed that the results were not good enough.

In [24]:
# Command for training torchvision ResNet50
#!python train.py train.epochs=20 train.batch_size=256 model.module=torchvision model.arch=resnet50

### ResNet50 pretrained

For the third try the pretrained version of ResNet50 network was used. The pretraining was done on the larger Imagenet dataset. 

In [ ]:
# Command for training ResNet50 pretrained on Imagnet
#!python train.py train.epochs=20 train.batch_size=256 model.module=torchvision model.arch=resnet50 model.pretrained=true

In [ ]:
# Download results from runtime
#!zip -r /kaggle/working/results.zip /kaggle/working/tinyimagenet/results

## Evaluate

In [ ]:
import argparse
import logging
from typing import Tuple, Union

#import albumentations as albu
import numpy as np
import torch
from hydra.utils import instantiate, to_absolute_path
from omegaconf import OmegaConf
from torch.nn.modules import loss
from torch.utils.data import DataLoader

from modules.dataset import DatasetItem, TinyImagenetDataset
from modules.runner import test
from modules.transform import to_tensor_normalize
from torchvision import models
import torch.nn as nn

In [ ]:
def evaluate_model(
    results_root: Union[str, Path], data_part: str = "val", device: str = "cuda"
) -> Tuple[float, float, np.ndarray]:
    """
    The main training function
    :param results_root: path to results folder
    :param data_part: {train, val, test} partition to evaluate model on
    :param device: {cuda, cpu}
    :return: None
    """
    results_root = Path(results_root)
    logging.basicConfig(
        filename=results_root / f"{data_part}.log", level=logging.NOTSET
    )
    # Setup logging and show config ьфлу
    log = logging.getLogger(__name__)
    if not log.handlers:
        log.addHandler(logging.StreamHandler())

    cfg_path = results_root / ".hydra/config.yaml"
    log.info(f"Read config from {cfg_path}")

    cfg = OmegaConf.load(str(cfg_path))
    log.info(f"Config:\n{OmegaConf.to_yaml(cfg)}")

    # Specify results paths from config
    checkpoint_path = results_root / cfg.results.checkpoints.root
    checkpoint_path /= f"{cfg.results.checkpoints.name}.pth"

    # Data
    # Specify data paths from config
    data_root = Path(cfg.data.root)
    test_path = data_root / data_part

    # Check if dataset is available
    log.info(f"Looking for dataset in {str(data_root)}")
    if not data_root.exists():
        log.error(
            "Folder not found. Terminating. "
            "See README.md for data downloading details."
        )
        raise FileNotFoundError

    valid_transform = to_tensor_normalize()
    if "augmentation" in cfg:
        pre_transform = albu.load(
            to_absolute_path(cfg.augmentation.pre), data_format="yaml"
        )
        post_transform = albu.load(
            to_absolute_path(cfg.augmentation.post), data_format="yaml"
        )
        valid_transform = albu.Compose([pre_transform, post_transform])

    test_dataset = TinyImagenetDataset(test_path, cfg.data, valid_transform)
    test_loader = DataLoader(
        test_dataset,
        batch_size=cfg.train.batch_size,
        shuffle=False,
        collate_fn=DatasetItem.collate,
        num_workers=cfg.train.num_workers,
        pin_memory=True,
        drop_last=False,
    )

    log.info(
        f"Created test dataset ({len(test_dataset)}) "
        f"and loader ({len(test_loader)}): "
        f"batch size {cfg.train.batch_size}, "
        f"num workers {cfg.train.num_workers}"
    )

    loss_function = loss.CrossEntropyLoss()
    #model = instantiate(cfg.model)
    model = models.resnet50()
    model.fc = nn.Linear(2048, 200)
    try:
        
        model.load_state_dict(torch.load(checkpoint_path, map_location="cpu"))
    except RuntimeError as e:
        log.error("Failed loading state dict")
        raise e
    except FileNotFoundError as e:
        log.error("Checkpoint not found")
        raise e
    log.info(f"Loaded model from {checkpoint_path}")
    device = (
            torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        )
    test_loss, test_acc, test_outputs = test(
        model, device, test_loader, loss_function, 0, log
    )
    log.info(f"Loss {test_loss}, acc {test_acc}")
    log.info(f"Outputs:\n{test_outputs.shape}\n{test_outputs[:5, :5]}")
    logging.shutdown()
    return test_loss, test_acc, test_outputs


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("-r", "--results", help="results root", type=str)
  #  parser.add_argument(
   #     "-p",
   #     "--data_part",
   #     default="val",
   #     choices=["train", "val", "test"],
   #     help="data partition to evaluate on",
   #     type=str,
   # )
    #parser.add_argument(
    #    "-d", "--device", default="cuda", choices=["cuda", "cpu"], type=str
    #)
    args = parser.parse_args()
    test_loss, test_acc, test_outputs = evaluate_model(
        args.results
    )

In [131]:
test_loss_v1, test_acc_v1, test_outputs_v1 = evaluate_model('results/torchvision.resnet50/2021-11-26_07-51-03')

In [79]:
test_loss_v2, test_acc_v2, test_outputs_v2 = evaluate_model('results/torchvision.resnet50/2021-11-26_09-09-36')

## Results

For two of the tries evauation was used and the result follow. The tensorboard outputs for three networks are shown in graph below. It seem that two of the network were good enough to best the accuracy limit of 55%.

In [ ]:
import seaborn as sns
from scipy.special import softmax
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from functools import reduce
import pandas as pd

In [82]:
print('ResNet50 acc:', test_acc_v1)
print('ResNet50 pretrained acc:', test_acc_v2)

In [100]:
def read_tensorboard(path, tags=['train_acc', 'test_acc']):
    ea =  EventAccumulator(path)
    ea.Reload()
    dfs = []
    for tag in tags:
        df = pd.DataFrame(ea.Scalars(tag)).drop('wall_time', 1)
        df.columns = ['step', tag]
        dfs.append(df)
    dfs = reduce(lambda left, right: pd.merge(left, right, on='step'), dfs)
    return dfs

In [107]:
training_v1 = read_tensorboard('/kaggle/working/tinyimagenet/results/torchvision.resnet50/2021-11-26_07-51-03/tensorboard/events.out.tfevents.1637913071.54e8db379f4c.558.0')
training_v2 = read_tensorboard('/kaggle/working/tinyimagenet/results/torchvision.resnet50/2021-11-26_09-09-36/tensorboard/events.out.tfevents.1637917781.54e8db379f4c.4350.0')
training_v3 = read_tensorboard('/kaggle/working/tinyimagenet/results/models.resnet18/2021-11-19_16-35-06/tensorboard/events.out.tfevents.1637339715.d3a5626814b4.1626.0')

training = pd.concat([
    training_v1.assign(model='resnet50'),
    training_v2.assign(model='resnet50_pre'),
    training_v3.assign(model='resnet18_v2')
])
training = training.melt(id_vars=['step', 'model'], value_name='accuracy', var_name='split')

training.split = training.split.str.replace('_acc', '')

training.head()

In [129]:
sns.lineplot(x='step', y='accuracy', hue='model', style='split', data=training)
plt.plot([0, 40], [55, 55], label="55%", color="red")
plt.show()

In [110]:
cfg = OmegaConf.load('config/data/tinyimagenet.yaml')
print(OmegaConf.to_yaml(cfg))

In [113]:
data_root = Path(cfg.root)
val_path = data_root / cfg.val
val_dataset = dataset.TinyImagenetDataset(val_path, cfg, None)

In [115]:
from sklearn.metrics import confusion_matrix

cm_v1 = confusion_matrix(y_true=val_dataset._df.label,
                         y_pred=np.argmax(test_outputs_v1, 1))

cm_v2 = confusion_matrix(y_true=val_dataset._df.label,
                         y_pred=np.argmax(test_outputs_v2, 1))

In [132]:
fig, axes = plt.subplots(ncols=2, figsize=(9, 4))
for cm, ax, t in zip([cm_v1, cm_v2], axes, ['Resnet50', 'Resnet50_pretrained']):
    sns.heatmap(cm, ax=ax, vmax=50)
    ax.set_title(t)
plt.tight_layout()
plt.show()

In [118]:
test_pred_v2 = softmax(test_outputs_v2, 1)
sorted_by_correct_score = np.argsort(test_pred_v2[np.arange(len(test_pred_v2)), val_dataset._df.label.tolist()])
sorted_by_correct_score[:5]

In [120]:
folders_to_num, val_labels = dataset.get_labels_mapping(cfg)

class_labels = pd.read_csv(data_root / cfg.train_labels, sep='\t', header=None)
class_labels[0] = class_labels[0].apply(lambda x: int(folders_to_num[x]) if x in folders_to_num else None)
class_labels = class_labels.dropna()

class_labels_dict = {int(r[0]): r[1].split(',')[0] for i, r in class_labels.iterrows()}

In [121]:
def clean_show(ax):
    "Show plt figure without axes in tight layout"
    plt.setp(ax, xticks=[], yticks=[])
    plt.tight_layout()
    plt.show()

In [122]:
def show_examples(dataset, ids, predictions, s=2):
    n_examples = len(ids)
    fig, axes = plt.subplots(ncols=n_examples, figsize=(s*n_examples, s))
    for ax, i in zip(axes, ids):
        item = dataset[i]
        ax.imshow(item.image)
        ax.set_title(item.id)
        pred_label = np.argmax(predictions[i])
        label = f"T: {class_labels_dict[item.label]} ({predictions[i, item.label]:.2e})\n" \
                f"P: {class_labels_dict[pred_label]} ({predictions[i, pred_label]:.2e})"
        ax.set_xlabel(label)
    clean_show(axes)

In [123]:
show_examples(val_dataset, sorted_by_correct_score[:7], test_pred_v2)

In [124]:
def class_accuracy(predictions, labels):
    accuracy = {}
    for lab in np.unique(labels):
        mask = labels == lab
        accuracy[class_labels_dict[lab]] = np.mean(np.argmax(predictions, 1)[mask] == labels[mask])
    return pd.DataFrame(accuracy, index=['accuracy']).T

In [125]:
class_accuracy_df = class_accuracy(test_pred_v2, val_dataset._df.label)

In [126]:
class_accuracy_df.sort_values('accuracy').head(5)

In [127]:
class_accuracy_df.sort_values('accuracy', ascending=False).head(5)